In [69]:
# https://kyso.io/helenadomo/earthquake1-2#code=output

In [70]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

import numpy as np
import pandas as pd
import csv
import os



In [8]:
def read_img_ids_from_file(filename):
    """
    reads the the image ids from preselected 2000 image ids
    """
    img_ids = []
    with open(filename) as file:
        for line in file:
            line = line.strip().split("\n")
            line = "".join(str(char) for char in line)
            img_ids.append(line)

    return img_ids

In [9]:
filepath="../fashion-product-images-small/images/"
filename = 'smaller_chunk_of_samples.txt'
# all_image_ids =  get_all_image_ids(filepath)
part_of_all_images = read_img_ids_from_file(filename)

In [10]:
print(len(part_of_all_images))

2000


In [64]:
def cosine_sim_gen(all_ids):
    line = 0
    with open('cos_sim_table_3.txt', 'a') as file:
        counter = 0
        for i in range(len(all_ids)):
#             for j in range(i+1, len(all_ids)):
            
            for j in range(len(all_ids)):
                image_i = all_ids[i]
                image_j = all_ids[j]
                image_i = preprocess_image_for_vgg(filepath, image_i)
                image_j = preprocess_image_for_vgg(filepath, image_j)
                shape_i = model.predict(image_i).reshape(1, -1)
                shape_j = model.predict(image_j).reshape(1, -1)
                
#                 yield (all_ids[i], all_ids[j], cosine_similarity(shape_i, shape_j)[0][0])
#                 cos_sim = cosine_similarity(shape_i, shape_j)
                score = str(cosine_similarity(shape_i, shape_j)[0][0])
#                 print(score)
                counter += 1
                if counter == len(all_ids):
                    file.write(score+'\n')
#                     print("finished line")
#                     print("line: ", line)
                    line += 1
                    
                    counter = 0
                else:
                    file.write(score+' ')
#                 print(counter)

In [65]:

# print(img_features.shape)
# # print(img_features.reshape(1,-1))

In [67]:
# this runs cos similarty among all images
# cosine_sim_gen(part_of_all_images)


### notes on that, it took me 3 hours just to create a file with 13k line of rows and i calculated that it would never able to finish the process for 2 billion images

In [4]:
# more_stuff = [img_features.reshape(1,-1), img_features.reshape(1,-1)]
# # test_cos = create_cosine_table(['a'], more_stuff)
# # print(test_cos)
# cosine_similarity(more_stuff)
# # print(img_features.reshape(1,-1))



In [63]:
df = pd.read_csv('cos_sim_table_3.txt', sep=" ", header=None)
# df.columns = ["image_i_id", 'image_j_id', 'cosine']
df.head()

,0,1,2,3,4
0,1.000000,0.195854,0.165746,0.201993,0.106626
1,0.195854,1.000000,0.300624,0.182000,0.157747
2,0.165746,0.300624,1.000000,0.199001,0.184345
3,0.201993,0.182000,0.199001,1.000000,0.138449
4,0.106626,0.157747,0.184345,0.138449,1.000000


In [86]:
csv_path = 'styles.csv'

In [90]:
style_df = pd.read_csv(csv_path)
style_df.head()


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,Unnamed: 10,Unnamed: 11
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,NaN,NaN
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,NaN,NaN
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,NaN,NaN
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,NaN,NaN
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,NaN,NaN


In [111]:
all_article_hist = {}
for pro_type in style_df['articleType']:
    if pro_type in all_article_hist:
        all_article_hist[pro_type] += 1
    else:
        all_article_hist[pro_type] = 1
# print(all_article_hist)
# print(sorted(all_article_hist, reverse=False))      
# style_df['articleType'].value_counts()
# there are 143 different types of products

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [16]:
def get_cosine_similarity(v1, v2):
    result = cosine_similarity(v1, v2)
    return result

In [9]:
# a = [[2, 0, 1, 1, 0, 2, 1, 1]]

# b = [[2, 1, 1, 0, 1, 1, 1, 1]]
# img_features = img_features.reshape(1, 7*7*512)
# # print("yhat shape->", yhat.shape)

# # print(get_cosine_similarity(a, a))
# print("yhat cos sim: ")
# # print(img_features)
# print(get_cosine_similarity(img_features, img_features))

In [17]:
def preprocess_image_for_vgg(filepath, img_id):
    """
    Takes an image and returns 4D array of features from vgg16 model
    
    Args:
        image_path: path to image
    """
    # load an image from file
    original_img = load_img(filepath + img_id, target_size=(224, 224))
    # convert the image pixels to a numpy array
    nparry_img = img_to_array(original_img)
    # # prepare the image for the VGG model
    image = np.expand_dims(nparry_img, axis=0)
    # print("image shape:", image.shape)
    processed_image = preprocess_input(image)
#     print("preprocess")
#     print(processed_image)
    # predict the probability across all output classes
    
    return processed_image


def get_feature_vector_from_vgg(processed_image):
    # dont use include top for vgg, because we dont want mlp part
    model = VGG16(weights='imagenet', include_top=False)
    
    feature_vector = model.predict(processed_image)
    return feature_vector



In [11]:
a = [[2, 0, 1, 1, 0, 2, 1, 1]]
b = [[2, 1, 1, 0, 1, 1, 1, 1]]
c = [[2, 3, 6, 1, 0, 2, 8, 0]]
test_for_cos_df = a + b + c
print(test_for_cos_df)

[[2, 0, 1, 1, 0, 2, 1, 1], [2, 1, 1, 0, 1, 1, 1, 1], [2, 3, 6, 1, 0, 2, 8, 0]]


In [18]:
def get_all_image_ids(filepath):
    images_ids = []
    # listing all the image names into list they are ids of the image
    for img in os.listdir(filepath):
        if img.endswith('jpg'):
            images_ids.append(img)
    
    return images_ids

# reshape the feature_vector so we can find cosine similarity score
def reshape_feature_vector(feature_vector):
    """
    Takes 4d array, return 2D representation
    """
    # this is the vgg output 7, 7, 512
    return feature_vector.reshape(1, 7*7*512)
    
# print(reshape_feature_vector(img_features))

In [19]:
# make vgg_feature list 
def list_of_vgg_features(all_imgs):
    # creates the list of vgg features and reshape it ready for making cosine sim score later
    all_feature_vectors = []
    
    for index, img in enumerate(all_imgs):
        feature_vector = get_feature_vector_from_vgg(img)
        # reshape the feature vector
        feature_vector = reshape_feature_vector(feature_vector)
        
        all_feature_vectors.append(feature_vector)
        
    # return all feature vectors
    return all_imgs, all_feature_vectors
        

In [20]:
# def get_cosine_similarity(v1, v2):
#     result = cosine_similarity(v1, v2)
#     return result

In [21]:
def create_cosine_table(list_of_ids, list_of_features):
    """Creates a cosine similarity table from list of vectors"""
    cos_sims = cosine_similarity(list_of_features)
    print(cos_sims)
    cos_table = pd.DataFrame(cos_sims, columns=list_of_ids, index=list_of_ids)
    # turn the cos sim into dataframe
    return cos_table

In [22]:
def get_cosine_table(list_vector):
    cos_list = []
    for i in range(len(list_vector)):
        x = np.array(list_vector[i]).reshape(1, -1)
        for j in range(len(list_vector)):
            y = np.array(list_vector[j]).reshape(1, -1)
            score = cosine_similarity(x, y)
            cos_list.append(score)
    
    df_col = ["a", "b", "c"]
    idx = ["a", "b", "c"]
    print(np.array(cos_list))
    cos_df = pd.DataFrame(cos_list, columns=df_col, index=idx)
    cos_df.head()
    print(cos_list)
    


# #     cos_df.sort_values(by=df_col, axis=1, ascending=False, inplace=False, kind='quicksort', na_position='last')
#     return cos_df
# test_cos_table(test_for_cos_df)

In [26]:
# test_cos_table = test_cos_table(test_for_cos_df)
# test_cos_table

In [27]:
def get_k_most_similar_to(df, given_img, k_similar=5):
    # this gets the rows
    k_images = df[given_img].sort_values(ascending=False)[1:k_similar+1].index
    # this get the corresod=nding scores
    k_images_score = df[given_img].sort_values(ascending=False)[1:k_similar+1]
    for img in range(len(k_images)):
        original = k_images[i]
        plt.imshow(original)
        plt.show()
        print("Similarity score:", k_images_score[i])

In [28]:
def generate_vectors(iterable):
    for item in iterable:
        yield item

In [20]:
# filepath="../fashion-product-images-small/images/"
# list_images = get_all_image_ids(filepath)


In [21]:
# all_image_generator = generate_vectors(list_images)

In [20]:
# all_processed_images = []
# for i, image in enumerate(all_image_generator):
#     processed_image = preprocess_image_for_vgg(filepath, image)
#     if i == 40000:
#         print(processed_image)
#     all_processed_images.append(processed_image)
        
# # print(all_processed_images)

In [21]:
# len(all_processed_images)

In [22]:
# # now make a vgg vectors
# all_vgg_features = []
# vgg_generator = generate_vectors(all_processed_images)
# for image in vgg_generator:
#     feature_vector = get_feature_vector_from_vgg(image)
#     # reshape it before we put into feature vector
#     feature_vector = reshape_feature_vector(feature_vector)
#     # now append it to all features list
#     all_vgg_features.append(feature_vector.flatten())
    

In [ ]:
len(all_vgg_features)

In [30]:
def run_all(filepath):
    print("======================STARTING============================")
    all_images = get_all_image_ids(filepath)
    print("some examples: ", all_images[:20])
    # we preprocess the image to make it ready to pass to vgg model
    print("prepocess")
    all_processed_images = []
#     all_images = np.random.choice(all_images, 5)
#     all_images = all_images[:5]
#     gen = generate_vectors(all_images)
    for image in all_images:
        processed_image = preprocess_image_for_vgg(filepath, image)
        all_processed_images.append(processed_image)
        
    print("Now we have preprocessed image for vgg")
    all_feature_vectors = []
    for image in all_processed_images:
        # create feature vector
        feature_vector = get_feature_vector_from_vgg(image)
#         print("making vgg")
        # reshape it before we put into 
#         print("feature shape:", feature_vector.shape)
        feature_vector = reshape_feature_vector(feature_vector)
#         print("after reshaping", feature_vector)
        
        # now append it to all features list
        all_feature_vectors.append(feature_vector.flatten())
    
    print("Now we have all feature vector!")
    all_feature_vectors = np.array(all_feature_vectors)
    print("shape of the feature vector")
    print(all_feature_vectors.shape)
#     create cos sim table

    
    print("we have list of features ready to pass cosine similarity!")
    print("passing...")
    cos_df = create_cosine_table(all_images, all_feature_vectors)
    if len(cos_df) < 2000:
        cos_df.to_csv('cos_similarity_small_table.csv', sep='\t', encoding='utf-8')
    else:
        cos_df.to_csv('cos_similarity_table.csv', sep='\t', encoding='utf-8')
    return cos_df
#     return all_feature_vectors

In [84]:
# filepath = "../fashion-product-images-small/images/"
# result = run_all(filepath)
# # print(result.shape)


In [24]:
result['9733.jpg'].sort_values(ascending=False)[:3].index
result['9733.jpg'].sort_values(ascending=False)[:3]

9733.jpg     1.000000
52112.jpg    0.289882
34297.jpg    0.281507
Name: 9733.jpg, dtype: float32

In [25]:
result

,9733.jpg,14147.jpg,52112.jpg,6400.jpg,34297.jpg
9733.jpg,1.000000,0.141262,0.289882,0.146407,0.281507
14147.jpg,0.141262,1.000000,0.149685,0.302827,0.185416
52112.jpg,0.289882,0.149685,1.000000,0.160585,0.397643
6400.jpg,0.146407,0.302827,0.160585,1.000000,0.178366
34297.jpg,0.281507,0.185416,0.397643,0.178366,1.000000


In [122]:
# all_images = get_all_image_ids(filepath)
# smaller_chunk_of_images = np.random.choice(all_images, 2000)

In [123]:
# with open('smaller_chunk_of_samples.txt', 'a') as file:
#     for img_id in smaller_chunk_of_images:
#         file.write(img_id+'\n')
# import sys
# import numpy
# numpy.set_printoptions(threshold=sys.maxsize)

# (smaller_chunk_of_images)